### PageRank as a linear algebra problem
Let's imagine a micro-internet, with just 4 websites (**A**, **B**,**C**,**D**)

The design principle of PageRank is that important websites will be linked to by important websites.
This somewhat recursive principle will form the basis of the program.

Imagine we have 100 **Sheldons'** on our micro-internet, each viewing a single website at a time.
Each minute the **Sheldons** follow a link on their website to another site on the micro-internet.

After a while, the websites that are most linked to will have more **Sheldon** visiting them, and in the long run, each minute for every **Sheldon** that leaves a website, another will enter keeping the total numbers of **Sheldon** on each website constant.
The PageRank is simply the ranking of websites by how many Sheldon they have on them at the end of this process.

We represent the number of **Sheldon** on each website with the vector,
$$\mathbf{r} = \begin{bmatrix} r_A \\ r_B \\ r_C \\ r_D \\  \end{bmatrix}$$
And say that the number of Sheldon on each website in minute $i+1$ is related to those at minute $i$ by the matrix transformation

$$ \mathbf{r}^{(i+1)} = L \,\mathbf{r}^{(i)}$$
with the matrix $L$ taking the form,
$$ L = \begin{bmatrix}
L_{A→A} & L_{B→A} & L_{C→A} & L_{D→A}\\
L_{A→B} & L_{B→B} & L_{C→B} & L_{D→B}\\
L_{A→C} & L_{B→C} & L_{C→C} & L_{D→C}\\
L_{A→D} & L_{B→D} & L_{C→D} & L_{D→D}\\
\end{bmatrix}
$$
where the columns represent the probability of leaving a website for any other website, and sum to one.
The rows determine how likely you are to enter a website from any other, though these need not add to one.
The long time behaviour of this system is when $ \mathbf{r}^{(i+1)} = \mathbf{r}^{(i)}$, so we'll drop the superscripts here, and that allows us to write,
$$ L \,\mathbf{r} = \mathbf{r}$$

which is an eigenvalue equation for the matrix $L$, with eigenvalue 1 (this is guaranteed by the probabalistic structure of the matrix $L$).



In [1]:
import numpy.linalg as la
import numpy as np
L=np.array([[0,0.5,0,0],
            [0.33,0,0,0.5],
            [0.33,0,0,0.5],
            [0.33,0.5,1,0]])
L

array([[0.  , 0.5 , 0.  , 0.  ],
       [0.33, 0.  , 0.  , 0.5 ],
       [0.33, 0.  , 0.  , 0.5 ],
       [0.33, 0.5 , 1.  , 0.  ]])

In [28]:
eVals, eVecs = la.eig(L) # Gets the eigenvalues and vectors
order = np.absolute(eVals).argsort()[::-1] # Orders them by their eigenvalues
eVals = eVals[order]
eVecs = eVecs[:,order]

r = eVecs[:, 0] # Sets r to be the principal eigenvector
100 * np.real(r / np.sum(r))

array([12.01115733, 23.99346109, 23.99346109, 40.00192049])

In [29]:
eVecs

array([[-0.22320352,  0.26221488, -0.8278286 , -0.8046482 ],
       [-0.44587085, -0.47608349,  0.15064355, -0.        ],
       [-0.44587085, -0.47608349,  0.15064355,  0.26553391],
       [-0.74335629,  0.69132653,  0.51895362,  0.53106781]])

We can see from this list, the number of Sheldon that we expect to find on each website after long times.
Putting them in order of *popularity* (based on this metric), the PageRank of this micro-internet is:

**C**, **D**, **A**, **B**

First let's set up our initial vector, $\mathbf{r}^{(0)}$, so that we have our 100  Sheldon equally distributed on each of our 4 websites.

In [30]:
r = 100 * np.ones(4) / 4 # Sets up this vector (4 entries of 1/4 × 100 each)
r # Shows it's value

array([25., 25., 25., 25.])

In [31]:
r = L @ r # Apply matrix L to r
r # Show it's value
# Re-run this cell multiple times to converge to the correct answer.

array([12.5 , 20.75, 20.75, 45.75])

In [32]:
r = 100 * np.ones(4) / 4 # Sets up this vector (4 entries of 1/4 × 100 each)
lastR = r
r = L @ r
i = 0
while la.norm(lastR - r) > 0.1 :
    lastR = r
    r = L @ r
    i += 1
print(str(i) + " iterations to convergence.")
r

54 iterations to convergence.


array([11.23875695, 22.41162654, 22.41162654, 37.42244384])

### Setting up a damping parameter

The system we just studied converged fairly quickly to the correct answer.
Let's consider an extension to our micro-internet where things start to go wrong.

$$ M = d \, L + \frac{1-d}{n} \, J $$

Say a new website is added to the micro-internet: **E** Website.
This website is linked to by **C** and only links to itself.



In [33]:
L2 = np.array([[0,1/2,1/3, 0,0],
               [1/3,0,0,0, 1/2],
               [1/3, 1/2, 0,1,0],
               [1/3,0,1/3,0,1/2],
               [0,0,0,0,0]])

In [34]:
r = 100 * np.ones(5) / 5 # Sets up this vector (6 entries of 1/6 × 100 each)
lastR = r
r = L2 @ r
i = 0
while la.norm(lastR - r) > 0.1 :
    lastR = r
    r = L2 @ r
    i += 1
print(str(i) + " iterations to convergence.")
r

29 iterations to convergence.


array([0.21872558, 0.08532795, 0.4326708 , 0.25412119, 0.        ])

In [35]:
d = 0.5
M = d * L2 + (1-d)/5 * np.ones([5,5])

In [36]:
r = 100 * np.ones(5) / 5 # Sets up this vector (6 entries of 1/6 × 100 each)
lastR = r
r = M @ r
i = 0
while la.norm(lastR - r) > 0.03 :
    lastR = r
    r = M @ r
    i += 1
print(str(i) + " iterations to convergence.")
r

85 iterations to convergence.


array([0.24364588, 0.20267903, 0.36295249, 0.26637608, 0.1265945 ])

## creating a customised internet page rank

In [37]:
import numpy as np
import numpy.linalg as la
np.set_printoptions(suppress=True)

In [2]:
import numpy as np
import numpy.linalg as la
np.set_printoptions(suppress=True)

## function to generate a randomised internet with n number of websites
def generate_internet(n) :
    c = np.full([n,n], np.arange(n))
    c = (abs(np.random.standard_cauchy([n,n])/2) > (np.abs(c - c.T) + 1)) + 0
    c = (c+1e-10) / np.sum((c+1e-10), axis=0)
    return c

## finds out the page rank with the link probability matrix with a damping factor of d. 
## d equals to 1 indicate a regular link matrix
def pageRank(linkMatrix, d) :
    n = linkMatrix.shape[0]
    M = d * linkMatrix + (1-d)/n * np.ones([n, n])
    r = 100 * np.ones(n) / n # Sets up this vector (n entries of 1/n × 100 each)
    last = r
    r = M @ r
    while la.norm(last - r) > 0.01 :
        last = r
        r = M @ r
    return r

In [3]:
a=10
q=generate_internet(a)
q

array([[0. , 0. , 0.1, 0.1, 0. , 0. , 0. , 0. , 0. , 0.1],
       [0. , 0. , 0.1, 0.1, 0. , 0.5, 0. , 0. , 0. , 0.1],
       [0. , 0. , 0.1, 0.1, 0. , 0. , 0. , 0. , 0. , 0.1],
       [0. , 0. , 0.1, 0.1, 0. , 0. , 0. , 0. , 0. , 0.1],
       [1. , 0.5, 0.1, 0.1, 1. , 0. , 0. , 0. , 0. , 0.1],
       [0. , 0. , 0.1, 0.1, 0. , 0.5, 0. , 0. , 0. , 0.1],
       [0. , 0.5, 0.1, 0.1, 0. , 0. , 1. , 0. , 0. , 0.1],
       [0. , 0. , 0.1, 0.1, 0. , 0. , 0. , 0. , 0. , 0.1],
       [0. , 0. , 0.1, 0.1, 0. , 0. , 0. , 1. , 0. , 0.1],
       [0. , 0. , 0.1, 0.1, 0. , 0. , 0. , 0. , 1. , 0.1]])

In [4]:
eVals, eVecs = la.eig(q) # Gets the eigenvalues and vectors
order = np.absolute(eVals).argsort()[::-1] # Orders them by their eigenvalues
eVals = eVals[order]
eVecs = eVecs[:,order]

r = eVecs[:, 0]
100 * np.real(r / np.sum(r))

array([ 0.00000002,  0.00000004,  0.00000002,  0.00000002, 59.99999316,
        0.00000004, 40.00000658,  0.00000002,  0.00000004,  0.00000006])

In [10]:
pageRank(q,0.4)

array([ 6.99636442,  8.74567315,  6.99636442,  6.99636442, 19.23872194,
        8.74567315, 14.57483319,  6.99636442,  9.79502521, 10.91461569])